## Step 1: Remove Useless columns
In this step, we load all Shopify order files and combine them into a single DataFrame. We then remove columns that are either not useful for modeling (e.g., customer addresses or always-null fields) or columns that are filled in less than 5% of the rows. This results in a cleaner dataset with fewer missing values and less noise, which is essential for downstream modeling tasks.

In [ ]:
import pandas as pd
import os

RAW_DATA_PATH = "raw_data"
PROCESSED_DATA_PATH = "processed_data"
output_path = os.path.join(PROCESSED_DATA_PATH, "orders_cleaned_columns.csv")

# File paths
order_files = [
    os.path.join(RAW_DATA_PATH, "shopify_orders", "orders_export_1.csv"),
    os.path.join(RAW_DATA_PATH, "shopify_orders", "orders_export_2.csv"),
    os.path.join(RAW_DATA_PATH, "shopify_orders", "orders_export_3.csv"),
]

##########step 1 all orders in one dataframe##########
# all orders tables in one dataframe
orders_df = pd.concat([pd.read_csv(file) for file in order_files], ignore_index=True)


################step 2 remove useless columns###############
not_useful_columns = [
    "Currency", # always EUR
    "Shipping", # always 0
    "Shipping Method", # always "PostNL or nothing"
    "Lineitem name", # we already have an item sku
    "Lineitem compare at price", # always empty
    "Lineitem requires shipping", # always TRUE
    "Lineitem taxable", # always TRUE
    "Lineitem fulfillment status", # we have a column that indicates if the order is fulfilled or not 
    "Billing Phone", # (customer phone number) private customer information & does not benifit the model
    "Shipping Name", # ( customer name ) private customer information & does not benifit the model
    "Shipping Street", # ( customer address ) private customer information & does not benifit the model
    "Shipping Address1", # ( customer address ) private customer information & does not benifit the model
    "Shipping Address2", # ( customer address ) private customer information & does not benifit the model
    "Shipping Company", # ( customer address ) private customer information & does not benifit the model
    "Shipping Zip", # ( customer address ) private customer information & does not benifit the model
    "Shipping Phone", # ( customer phone number ) private customer information & does not benifit the model
    "Notes", # Payment notes are not useful for the model
    "Note Attributes", # Payment notes are not useful for the model
    "Payment Reference", # Payment notes are not useful for the model
    "Vendor", # always Moodies Undies
    "Outstanding Balance", # Does not benifit the model
    "Tax 1 Name", # Does not benifit the model
    "Tax 1 Value", # Does not benifit the model
    "Tax 2 Name", # Does not benifit the model
    "Tax 2 Value", # Does not benifit the model
    "Tax 3 Name", # Does not benifit the model
    "Tax 3 Value", # Does not benifit the model
    "Tax 4 Name", # Does not benifit the model
    "Tax 4 Value", # Does not benifit the model
    "Tax 5 Name", # Does not benifit the model
    "Tax 5 Value", # Does not benifit the model
    "Phone", # ( customer phone number ) private customer information & does not benifit the model
    "Receipt Number", # Does not benifit the model
    "Payment Terms Name",
    "Next Payment Due At",
    "Payment References"
]

orders_df.drop(columns=not_useful_columns, inplace=True, errors="ignore")




orders_df.to_csv(output_path, index=False)
orders_df.head()

## Step 2: Combine Multiple Rows Into One Per Order
Shopify splits a single order into multiple rows if it contains multiple unique items. This step identifies such rows using the 'Email' and 'Name' (which is the id of an order) fields, and merges them by combining their line items into a single list. Additionally, duplicate orders are filtered to keep only the fulfilled ones. The result is a dataset where each order occupies one row.

In [ ]:
orders_file = os.path.join(PROCESSED_DATA_PATH, "orders_cleaned_columns.csv")
output_path = os.path.join(PROCESSED_DATA_PATH, "orders_have_one_row.csv")
orders_df = pd.read_csv(orders_file)

orders_df["Lineitems"] = orders_df.apply(
    lambda row: [{row["Lineitem sku"] if pd.notna(row["Lineitem sku"]) else "unknown": row["Lineitem quantity"]}],
    axis=1
)

rows_to_drop = []
for i in range(1, len(orders_df)):
    current_row = orders_df.loc[i]
    previous_row = orders_df.loc[i - 1]
    if pd.isna(current_row["Paid at"]) and current_row["Name"] == previous_row["Name"] and current_row["Email"] == previous_row["Email"]:
        prev_items = previous_row["Lineitems"]
        curr_items = current_row["Lineitems"]
        prev_items.extend(curr_items)
        orders_df.at[i - 1, "Lineitems"] = prev_items
        rows_to_drop.append(i)

orders_df.drop(index=rows_to_drop, inplace=True)
orders_df.reset_index(drop=True, inplace=True)

# Remove duplicated orders unless fulfilled
duplicate_names = orders_df["Name"][orders_df["Name"].duplicated(keep=False)]
filtered_orders_df = orders_df[
    ~orders_df["Name"].isin(duplicate_names) |
    (orders_df["Fulfillment Status"] == "fulfilled")
]
filtered_orders_df = filtered_orders_df.drop_duplicates(subset="Name", keep="first")
orders_df = filtered_orders_df.reset_index(drop=True)

orders_df.to_csv(output_path, index=False)
print(f"Updated file saved to: {output_path}")
orders_df.head()


## Step 3: Merge Klaviyo Customer Data
We merge the cleaned Shopify order data with customer profile data from Klaviyo using the 'Email' field. This ensures that we capture both customers who have made purchases and those who have not. The result is a dataset that combines behavioral (order-based) and demographic (profile-based) information.

In [ ]:
orders_file = os.path.join(PROCESSED_DATA_PATH, "orders_have_one_row.csv")
klaviyo_file = os.path.join(RAW_DATA_PATH, "Klaviyo_everyone_email.csv")
output_path = os.path.join(PROCESSED_DATA_PATH, "orders_with_customer_data.csv")

orders_df = pd.read_csv(orders_file)
customers_df = pd.read_csv(klaviyo_file)

merged_df = pd.merge(
    customers_df,
    orders_df,
    on="Email",
    how="outer",
    suffixes=("", "_order")
)

merged_df.to_csv(output_path, index=False)
print(f"Merged file with all customers saved to: {output_path}")
merged_df.head()


## Step 4: Remove columns that are filled for in less than 5 percent of the rows

In [ ]:

input_path = os.path.join(PROCESSED_DATA_PATH, "orders_with_customer_data.csv")
output_path = os.path.join(PROCESSED_DATA_PATH, "orders_with_customer_data_no_empty_columns.csv")

def get_fill_percentages(df: pd.DataFrame) -> dict:
    """
    Returns a dictionary with the percentage of filled (non-null and non-empty) rows per column.

    :param df: DataFrame to inspect.
    :return: Dictionary {column_name: fill_percentage}
    """
    total_rows = len(df)
    fill_percentages = {}

    for col in df.columns:
        non_empty = df[col].notna() & (df[col].astype(str).str.strip() != "")
        fill_percentage = (non_empty.sum() / total_rows) * 100
        fill_percentages[col] = round(fill_percentage, 2)

    return fill_percentages


orders_with_customer_df = pd.read_csv(input_path)
fill_stats = get_fill_percentages(orders_with_customer_df)

for col, pct in fill_stats.items():
    print(f"{col}: {pct}% filled")
    if pct < 5:
        print(f"{col} has a fill rate of {pct}% which is below the threshold of 95%. column {col} will be dropped.")
        orders_df.drop(columns=[col], inplace=True, errors="ignore")

## Step 5: Create Monthly Spend and Order Features
Here, we engineer new features for modeling. For each customer, we compute the number of orders and the amount spent in each month. These features are crucial for understanding customer behavior over time and are especially useful for predicting future value.

In [ ]:
orders_file = os.path.join(PROCESSED_DATA_PATH, "orders_have_one_row.csv")
output_file = os.path.join(PROCESSED_DATA_PATH, "monthly_customer_stats.csv")

orders_df = pd.read_csv(orders_file, low_memory=False)
orders_df["Paid at"] = pd.to_datetime(orders_df["Paid at"], utc=True, errors="coerce")
orders_df = orders_df.dropna(subset=["Paid at"])
orders_df["YearMonth"] = orders_df["Paid at"].dt.strftime("%Y_%m")

monthly_summary = orders_df.groupby(["Email", "YearMonth"]).agg(
    orders_per_month=("Name", "count"),
    amount_per_month=("Total", "sum")
).reset_index()

order_counts = monthly_summary.pivot(index="Email", columns="YearMonth", values="orders_per_month")
amounts_spent = monthly_summary.pivot(index="Email", columns="YearMonth", values="amount_per_month")

order_counts = order_counts.reindex(sorted(order_counts.columns), axis=1)
amounts_spent = amounts_spent.reindex(sorted(amounts_spent.columns), axis=1)

order_counts.columns = [f"orders_in_{col}" for col in order_counts.columns]
amounts_spent.columns = [f"amount_spent_{col}" for col in amounts_spent.columns]

monthly_stats = pd.concat([order_counts, amounts_spent], axis=1)
monthly_stats["total_amount_spent"] = orders_df.groupby("Email")["Total"].sum()

monthly_stats = monthly_stats.reset_index()
monthly_stats.to_csv(output_file, index=False)
print(f"Monthly customer stats saved to: {output_file}")
monthly_stats.head()


## Step 5: Merge Monthly Features Into Full Dataset
In this final preprocessing step, we merge the engineered monthly features into the combined Klaviyo + Shopify dataset. This gives us one complete table that includes customer profile data, order history, and time-based behavioral features, ready for training a Customer Lifetime Value prediction model.

In [ ]:
orders_customers_file = os.path.join(PROCESSED_DATA_PATH, "orders_with_customer_data.csv")
monthly_stats_file = os.path.join(PROCESSED_DATA_PATH, "monthly_customer_stats.csv")
output_file = os.path.join(PROCESSED_DATA_PATH, "final_merged_data.csv")

orders_customers_df = pd.read_csv(orders_customers_file)
monthly_stats_df = pd.read_csv(monthly_stats_file)

final_df = pd.merge(
    orders_customers_df,
    monthly_stats_df,
    on="Email",
    how="left"
)

final_df.to_csv(output_file, index=False)
print(f"Final merged dataset saved to: {output_file}")
final_df.head()


### Step 6 – Final Feature Selection and Aggregation

In this step, we finalized our customer-level dataset by cleaning, transforming, and aggregating features to prepare them for modeling.

#### Column Filtering
We removed over 70 columns that were either:
- Sensitive (e.g. names, addresses, phone numbers)
- Duplicated or irrelevant for modeling
- High-cardinality categorical columns unlikely to provide predictive power
- System-generated metadata or unused campaign tracking codes

#### Boolean Normalization
We identified object-type columns that semantically represented booleans (e.g., `InWelcomeFlow`, `InCheckoutAbandonmentFlow`, `Accepts Marketing`) and explicitly converted them to `True`/`False`, filling missing values as `False`.

#### Location Cleanup
Missing values in the `Country` column were assumed to be “Netherlands”, as that is the primary market. Missing `City`, `Last Source`, and `Campaign` fields were filled with `"unknown"` or `"(direct)"` where applicable.

#### Date Feature Engineering
We engineered customer timeline features based on:
- First and last activity
- First and last purchase
- Days since email marketing consent

This resulted in numerical columns like `days_since_first_active` and `customer_lifetime_days`, giving the model a richer temporal understanding of each customer.

#### Email Marketing Consent
Instead of treating `Email Marketing Consent Timestamp` as a raw date, we extracted:
- `email_marketing_optin`: binary indicator if consent was ever given
- `days_since_email_optin`: numeric time since consent (or -1 if not given)

This allowed us to convert a sparse timestamp into usable model features.

#### Lineitem Breakdown
Each row in the dataset originally represented a customer order and contained a `Lineitems` field — a list of `{sku: quantity}` dictionaries.
We:
- Counted total quantities (`total_amount_line_items_bought`)
- Extracted the first and second unique SKUs bought, plus their quantities (`first_item_sku`, `first_item_qty`, etc.)

This allowed us to preserve product-level information in a compact way, without introducing excessive sparsity.

####  Aggregation to Customer Level
As multiple orders could exist for a single customer, we grouped the dataset by `Email`, and:
- Summed all numeric fields (e.g. amount spent, quantity, days)
- Took the first occurrence of remaining non-numeric fields

This gave us one row per customer, suitable for training a model to predict their future value.


In [ ]:
import pandas as pd
import ast
from datetime import datetime
import os

input_file = os.path.join("processed_data", "final_merged_data.csv")
output_file = os.path.join("processed_data", "clv_model_data.csv")

# Load data
df = pd.read_csv(input_file)

# Drop irrelevant or sensitive columns
columns_to_drop = [
    'First Name', 'Last Name', 'Phone Number', 'Address', 'Address 2', 'Locale',
    'Locale: Country', 'Organization', 'Title', 'Birthday', 'birthday', '$birthday_source',
    'Billing Name', 'Billing Street', 'Billing Address1', 'Billing Address2', 'Billing Company',
    'Billing City', 'Billing Zip', 'Billing Province', 'Billing Country', 'Billing Province Name',
    'Shipping City', 'Shipping Province', 'Shipping Country', 'Shipping Province Name',
    'Email Suppressions', 'Email Suppressions Timestamp',
    'Email List Suppressions', 'Email List Suppressions Timestamp', 'Email List Suppressions Reasons',
    'Initial Referring Domain', 'Search Keyword', 'Initial Source Referrer',
    'Created at', 'Cancelled at', 'Payment Method', 'Payment ID', 'Id', 'Tags', 'Risk Level', 'Source',
    'Subtotal', 'Taxes', 'Total', 'Discount Code', 'Refunded Amount', 'Duties',
    'Lineitem quantity', 'Lineitem price', 'Lineitem sku', 'Lineitem discount',
    'Paid at', 'Fulfilled at', 'Financial Status', 'Accepts Marketing_order', 'undefined',
    'Profile Created On', 'Date Added', 'Last Open', 'Last Click',
    'Device ID', 'Location', 'Employee',
    'SMS Marketing Consent', 'SMS Marketing Consent Timestamp',
    'SMS Transactional Consent', 'SMS Transactional Consent Timestamp',
    'Total Customer Lifetime Value', 'Historic Customer Lifetime Value',
    'Historic Number Of Orders', 'Initial Source Content', 'Last Source Term',
    'Last Source First Page', 'Last Source Medium', 'Last Source Content', 
    '$timezone', 'CollectionPreference',
    'Fulfillment Status', 'Name', 'ZPD: Situation', 'VIP', 'UTM Source',
    'UTM Medium', 'ResendFulfilledOrderFlow', 'Survey 01/23',
    'T-LwHoyq: Swimwear: yes', 'UTM Campaign', 'UTM Content',
    'Last Source Referrer', 'Initial Source First Page', 'Initial Source Term',
    'Expected Date Of Next Order'
]
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Convert object-type booleans
bool_like_cols = ['InCheckoutAbandonmentFlow', 'InFulfilledOrderFlow', 'InWelcomeFlow', 'Accepts Marketing']
for col in bool_like_cols:
    df[col] = df[col].fillna(False).astype(bool)

# Fill defaults for important string fields
df["Country"] = df["Country"].fillna("Netherlands")
df["City"] = df["City"].fillna("unknown")
df["Last Source Campaign"] = df["Last Source Campaign"].fillna("unknown")
df["Last Source"] = df["Last Source"].fillna("unknown")
df["Initial Source Campaign"] = df["Initial Source Campaign"].fillna("(direct)")

# Parse Lineitems
df["Lineitems"] = df["Lineitems"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

# Convert dates
date_cols = ["First Active", "Last Active", "First Purchase Date", "Last Purchase Date"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

reference_date = datetime(2025, 4, 1)

# Feature engineering from dates
df["email_marketing_optin"] = df["Email Marketing Consent Timestamp"].notna().astype(int)
df["days_since_email_optin"] = (
    datetime(2025, 4, 1) - pd.to_datetime(df["Email Marketing Consent Timestamp"], errors="coerce")
).dt.days

# Fill missing with 0 or -1 depending on what makes more sense
df["days_since_email_optin"] = df["days_since_email_optin"].fillna(-1)
df.drop(columns=["Email Marketing Consent Timestamp"], inplace=True)

df["days_since_first_active"] = (reference_date - df["First Active"]).dt.days
df["days_since_last_active"] = (reference_date - df["Last Active"]).dt.days
df["days_since_first_purchase"] = (reference_date - df["First Purchase Date"]).dt.days
df["days_since_last_purchase"] = (reference_date - df["Last Purchase Date"]).dt.days
df["customer_lifetime_days"] = (df["Last Purchase Date"] - df["First Purchase Date"]).dt.days

# Drop original datetime columns
df.drop(columns=date_cols, inplace=True)

# Count total quantity of items in Lineitems
df["order_items_count"] = df["Lineitems"].apply(
    lambda items: sum(qty for item in items for qty in item.values()) if isinstance(items, list) else 0
)

# Aggregate to one row per customer
numeric_cols = df.select_dtypes(include='number').columns.tolist()
grouped_df = df.groupby("Email").agg({
    **{col: 'sum' for col in numeric_cols},
    **{col: 'first' for col in df.columns if col not in numeric_cols and col != "Email"}
}).reset_index()

# Rename engineered columns
grouped_df.rename(columns={
    "Discount Amount": "total_discount_amount",
    "order_items_count": "total_amount_line_items_bought"
}, inplace=True)

# ✨ Extract first, second, third item SKU and qty
def extract_item_and_qty(items, index):
    if isinstance(items, list) and len(items) > index:
        item = list(items[index].items())[0]
        return item[0], item[1]
    return None, 0

for i, label in enumerate(["first", "second"]):
    grouped_df[f"{label}_item_sku"] = grouped_df["Lineitems"].apply(lambda items: extract_item_and_qty(items, i)[0])
    grouped_df[f"{label}_item_qty"] = grouped_df["Lineitems"].apply(lambda items: extract_item_and_qty(items, i)[1])

# Save final dataset
grouped_df.to_csv(output_file, index=False)
print(f"Saved cleaned customer-level dataset to: {output_file}")


## Step 7: Create the Target Variable (CLV_next_3_months)

To train a model that predicts customer lifetime value (CLV) for the next 3 months, we first need to define our target.

### What we’re predicting:
We are summing the total amount a customer spent in the months of January, February, and March 2025. This becomes the column `CLV_next_3_months`, which will serve as our prediction target.

### Why this approach:
- Simple and interpretable
- Allows the model to focus on short-term CLV
- Aligns with business needs like quarterly revenue forecasting


In [ ]:
import pandas as pd
input_file = os.path.join("processed_data", "clv_model_data.csv")
output_file = os.path.join("processed_data", "clv_model_data_with_target.csv")

df = pd.read_csv(input_file)

# Define the target variable
target_months = ["amount_spent_2025_01", "amount_spent_2025_02", "amount_spent_2025_03"]
df["CLV_next_3_months"] = df[target_months].sum(axis=1)

# Drop future columns from features to avoid leakage
df.drop(columns=target_months, inplace=True)

df.to_csv(output_file, index=False)
print("Target variable created and future columns dropped.")


## Step 8: Preprocess Features for Modeling

Before training a machine learning model, we need to prepare our feature set.

### What we do:
- Drop identifier columns like `Email` which are not predictive
- Convert boolean columns into integer format (`True` → `1`, `False` → `0`)
- One-hot encode object/categorical columns for compatibility with XGBoost
- Replace special characters in column names to avoid parsing errors
- Fill remaining missing values with `0` to avoid training issues

In [ ]:
# Load data with target
input_file = os.path.join("processed_data", "clv_model_data_with_target.csv")
df = pd.read_csv(input_file)

# Drop identifiers
df = df.drop(columns=["Email"])

# Encode boolean columns as integers
bool_cols = df.select_dtypes(include=["bool"]).columns
df[bool_cols] = df[bool_cols].astype(int)

# Fill NaNs
df.fillna(0, inplace=True)

# Separate features and target
X = df.drop(columns=["CLV_next_3_months"])
y = df["CLV_next_3_months"].squeeze()

# Encode object/categorical columns using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

# Replace ALL non-alphanumeric characters with underscores
X.columns = X.columns.str.replace(r"[^a-zA-Z0-9_]", "_", regex=True)

## Step 9: Train/Test Split

To evaluate how well our model generalizes to unseen customers, we split our dataset into training and testing sets.

We use an 80/20 split, a common practice in supervised learning.


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
X_train_np = X_train.to_numpy()
y_train_np = np.asarray(y_train).ravel()  # 1D target
X_test_np = X_test.to_numpy()
y_test_np = np.asarray(y_test).ravel()

## Step 10: Baseline for CLV Prediction
Zero Spending: Predict 0 for all customers.

Global Average: Predict average of 3-month spend across all customers.

Customer-Specific Monthly Average × 3: Predict Total Amount Spent / # active months × 3.

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# True values
y_true = y_test_np

# 1. Zero spending baseline
y_zero = np.zeros_like(y_true)

# 2. Global average baseline
global_avg = y_train_np.mean()
y_global = np.full_like(y_true, global_avg)

# 3. Customer-specific monthly avg × 3 (only if available)
# For now, we simulate a per-customer average from training set
# You can replace this logic with actual per-customer monthly avg if available
y_personal_avg = np.full_like(y_true, (y_train_np.mean() / 3) * 3)

# Define a helper function
def print_metrics(name, y_pred):
    print(f"🔹 {name}")
    print("  RMSE:", round(mean_squared_error(y_true, y_pred), 2))
    print("  MAE :", round(mean_absolute_error(y_true, y_pred), 2))
    print("  R²  :", round(r2_score(y_true, y_pred), 4))

# Show results
print_metrics("Zero Spending Baseline", y_zero)
print_metrics("Global Average Baseline", y_global)
print_metrics("Customer Avg x 3 Baseline", y_personal_avg)

## Step 10: Choose, Train, and Evaluate Model

We will use **XGBoost Regressor**, a powerful and efficient model for tabular data.

### Why XGBoost:
- Performs well on structured data
- Handles missing values internally
- Provides feature importance insights
- Scales well for large datasets

We will train the model and evaluate its performance using:
- RMSE (Root Mean Squared Error)
- MAE (Mean Absolute Error)
- R² Score (explained variance)


In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import time

start = time.time()
model = XGBRegressor(random_state=42)
model.fit(X_train_np, y_train_np)

print(f"Training completed in {time.time() - start:.2f} seconds")

# Predict and evaluate
y_pred = model.predict(X_test_np)
print("RMSE:", root_mean_squared_error(y_test_np, y_pred))
print("MAE:", mean_absolute_error(y_test_np, y_pred))
print("R² Score:", r2_score(y_test_np, y_pred))

Great the model gets just above 85% accuracy.
## let's plot the results

In [ ]:
import matplotlib.pyplot as plt

# Model performance metrics
rmse = 22.36
mae = 2.12
r2 = 0.8547

# Labels and values
metrics = ['RMSE', 'MAE', 'R² Score']
values = [rmse, mae, r2]

# Create bar plot
plt.figure(figsize=(8, 5))
plt.bar(metrics, values, color='skyblue')
plt.title("Model Performance Metrics")
plt.ylabel("Score")
plt.ylim(0, max(values) + 5)

# Annotate values on bars
for i, v in enumerate(values):
    plt.text(i, v + 0.5, f"{v:.2f}", ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Predicted vs Actual Scatter Plot
# Shows how well predictions align with true values.
plt.figure(figsize=(6, 6))
plt.scatter(y_test_np, y_pred, alpha=0.3)
plt.plot([y_test_np.min(), y_test_np.max()], [y_test_np.min(), y_test_np.max()], 'r--')
plt.xlabel("Actual CLV")
plt.ylabel("Predicted CLV")
plt.title("Predicted vs Actual CLV")
plt.grid(True)
plt.tight_layout()
plt.show()